In [ ]:
import os
import pandas as pd
import numpy as np
import random
import configparser
import lap

def has_jump(seq):
    # print(seq.shape)
    return not ((seq[-1]- seq[0] + 1) == len(seq))

def batch_iou(bb1, bb2):
    bb1 = np.expand_dims(bb1, 1)
    bb2 = np.expand_dims(bb2, 0)
    xx1 = np.maximum(bb1[..., 0], bb2[..., 0])
    yy1 = np.maximum(bb1[..., 1], bb2[..., 1])
    xx2 = np.minimum(bb1[..., 2], bb2[..., 2])
    yy2 = np.minimum(bb1[..., 3], bb2[..., 3])
    w = np.maximum(0., xx2 - xx1)
    h = np.maximum(0., yy2 - yy1)
    wh = w * h
    o = wh / ((bb1[..., 2] - bb1[..., 0]) * (bb1[..., 3] - bb1[..., 1])                                      
        + (bb2[..., 2] - bb2[..., 0]) * (bb2[..., 3] - bb2[..., 1]) - wh)                                              
    return(o)

def assignment(cost_matrix, threshold=0.7):
    if cost_matrix.size == 0:
        return np.empty((0, 2), dtype=int), tuple(range(cost_matrix.shape[0])), tuple(range(cost_matrix.shape[1]))
    matches, unmatched_a, unmatched_b = [], [], []
    cost, x, y = lap.lapjv(cost_matrix, extend_cost=True, cost_limit=threshold)
    # cost, x, y = lap.lapjv(cost_matrix, extend_cost=True)
    for ix, mx in enumerate(x):
        if mx >= 0:
            matches.append([ix, mx])
    unmatched_a = np.where(x < 0)[0]
    unmatched_b = np.where(y < 0)[0]
    matches = np.asarray(matches)
    return matches.tolist(), unmatched_a.tolist(), unmatched_b.tolist()

def generate(root_dirs, det_dirs):
    for root, det_dir in zip(root_dirs, det_dirs):
        # dataset = pd.DataFrame([], columns=['frame', 'id', 'tg', 'lg', 'bg', 'rg', 'td', 'ld', 'bd', 'rd'])
        sequences = [os.path.join(root, d) for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))]
        det_sequences = [os.path.join(det_dir, d) for d in os.listdir(det_dir) if os.path.isdir(os.path.join(det_dir, d))]
        for gt_seq_path, det_seq_path in zip(sequences, det_sequences):
            # if gt_seq_path in ['../../.Datasets/DanceTrack/train\dancetrack0001',
            #                    '../../.Datasets/DanceTrack/train\dancetrack0002',
            #                    '../../.Datasets/DanceTrack/train\dancetrack0006',
            #                    '../../.Datasets/DanceTrack/train\dancetrack0008',
            #                    '../../.Datasets/DanceTrack/train\dancetrack0012',
            #                    '../../.Datasets/DanceTrack/train\dancetrack0015',]:
            #     continue
            datas = []
            gt_path = os.path.join(gt_seq_path, 'gt', 'gt.txt')
            det_path = os.path.join(det_seq_path, 'det.txt')
            if not os.path.exists(gt_path) or not os.path.exists(det_path):
                continue
            cfp = configparser.ConfigParser()
            cfp.read(os.path.join(gt_seq_path, 'seqinfo.ini'))
            gt_df = pd.read_csv(gt_path, header=None)
            gt_df.columns = ['frame', 'id', 't', 'l', 'b', 'r', 'conf', 'class', 'visibility']
            det_df = pd.read_csv(det_path, header=None)
            det_df.columns = ['frame', 't', 'l', 'b', 'r', 'conf']
            gt_df = gt_df.sort_values('frame')
            print(gt_seq_path)
            for frame in gt_df['frame'].unique():
                frame_gt_df = gt_df[gt_df['frame'] == frame]
                frame_gt_df = frame_gt_df[frame_gt_df['class'] == 1]
                frame_gt_df['b'] += frame_gt_df['t']
                frame_gt_df['r'] += frame_gt_df['l']
                frame_det_df = det_df[det_df['frame'] == frame]
                gt_bboxes = frame_gt_df[['t', 'l', 'b', 'r']].to_numpy()
                det_bboxes = frame_det_df[['t', 'l', 'b', 'r']].to_numpy()
                ious = batch_iou(gt_bboxes, det_bboxes)
                matches, _, _ = assignment(1 - ious, threshold=0.5)
                for i,j in matches:
                    datas.append(pd.DataFrame([[
                        frame,
                        frame_gt_df.iloc[i]['id'],
                        frame_gt_df.iloc[i]['t'],
                        frame_gt_df.iloc[i]['l'],
                        frame_gt_df.iloc[i]['b'],
                        frame_gt_df.iloc[i]['r'],
                        frame_det_df.iloc[j]['t'],
                        frame_det_df.iloc[j]['l'],
                        frame_det_df.iloc[j]['b'],
                        frame_det_df.iloc[j]['r'],
                    ]], columns=['frame', 'id', 'tg', 'lg', 'bg', 'rg', 'td', 'ld', 'bd', 'rd']))
                # if len(datas) > 100:
                #     break
            seq_name = gt_seq_path.split('\\')[1]
            dataset_name, split_name = gt_seq_path.split('\\')[0].split('/')[-2:]
            os.makedirs(f'{dataset_name}/{split_name}/{seq_name}/', exist_ok=True)
            dataset = pd.concat(datas)
            dataset.to_csv(f'{dataset_name}/{split_name}/{seq_name}/file.txt', header=None, index=None)
            # return
            # return dataset
    
generate(['../../.Datasets/DanceTrack/train',
          '../../.Datasets/DanceTrack/val',
          '../../.Datasets/MOT17/train',
          '../../.Datasets/MOT17/val',
          '../../.Datasets/MOT20/train',
          '../../.Datasets/MOT20/val',
          ], ['../../.Detections/DanceTrack/train',
          '../../.Detections/DanceTrack/val',
          '../../.Detections/MOT17/train',
          '../../.Detections/MOT17/val',
          '../../.Detections/MOT20/train',
          '../../.Detections/MOT20/val',
          ])


../../.Datasets/DanceTrack/train\dancetrack0016
../../.Datasets/DanceTrack/train\dancetrack0020
../../.Datasets/DanceTrack/train\dancetrack0023
../../.Datasets/DanceTrack/train\dancetrack0024
../../.Datasets/DanceTrack/train\dancetrack0027
../../.Datasets/DanceTrack/train\dancetrack0029
../../.Datasets/DanceTrack/train\dancetrack0032
../../.Datasets/DanceTrack/train\dancetrack0033
../../.Datasets/DanceTrack/train\dancetrack0037
../../.Datasets/DanceTrack/train\dancetrack0039
../../.Datasets/DanceTrack/train\dancetrack0044
../../.Datasets/DanceTrack/train\dancetrack0045
../../.Datasets/DanceTrack/train\dancetrack0049
../../.Datasets/DanceTrack/train\dancetrack0051
../../.Datasets/DanceTrack/train\dancetrack0052
../../.Datasets/DanceTrack/train\dancetrack0053
../../.Datasets/DanceTrack/train\dancetrack0055
../../.Datasets/DanceTrack/train\dancetrack0057
../../.Datasets/DanceTrack/train\dancetrack0061
../../.Datasets/DanceTrack/train\dancetrack0062
../../.Datasets/DanceTrack/train\dancetr

In [14]:
import pandas as pd
import numpy as np

df = pd.read_csv('../../.Datasets/MOT17/train/MOT17-02-FRCNN/gt/gt.txt', names=['frame', 'id', 't', 'l', 'b', 'r', 'conf', 'class', 'visibility'])
df['b'] += df['t']
df['r'] += df['l']


In [26]:
df = pd.read_csv('MOT17/train/MOT17-02-FRCNN/file.txt', names=['frame', 'id', 'tg', 'lg', 'bg', 'rg', 'td', 'ld', 'bd', 'rd'])


In [29]:
df['bg'].argmax()

np.int64(12530)

In [33]:
df[df['id'] == 16][['tg', 'lg', 'bg', 'rg', 'td', 'ld', 'bd', 'rd']].to_numpy()

array([[1800,  353, 2301, 1346, 1769,  290, 2223, 1455],
       [1747,  348, 2246, 1345, 1728,  286, 2219, 1763],
       [1694,  344, 2192, 1344, 1719,  272, 2216, 1379],
       [1641,  340, 2138, 1343, 1632,  266, 2145, 1325],
       [1588,  336, 2084, 1342, 1574,  328, 1965, 1264],
       [1535,  332, 2030, 1341, 1528,  312, 1946, 1291],
       [1482,  327, 1975, 1340, 1475,  278, 1948, 1528],
       [1429,  323, 1921, 1339, 1437,  294, 1856, 1358],
       [1376,  319, 1867, 1338, 1381,  282, 1895, 1487],
       [1323,  315, 1813, 1337, 1330,  288, 1831, 1499],
       [1270,  311, 1759, 1337, 1277,  302, 1755, 1476],
       [1221,  324, 1690, 1339, 1227,  313, 1672, 1436],
       [1173,  338, 1621, 1342, 1188,  346, 1586, 1370],
       [1124,  351, 1552, 1345, 1141,  359, 1532, 1358],
       [1076,  365, 1484, 1348, 1089,  348, 1471, 1347],
       [1029,  362, 1432, 1345, 1034,  373, 1416, 1390],
       [ 983,  359, 1381, 1342, 1001,  367, 1352, 1328],
       [ 936,  357, 1330, 1339,

In [30]:
df.iloc[12530]

frame     426
id         16
tg       1800
lg        353
bg       2301
rg       1346
td       1769
ld        290
bd       2223
rd       1455
Name: 12530, dtype: int64

In [20]:
df['b'].argmax()

np.int64(7310)

In [25]:
df[df['id'] == 16][['t', 'l', 'b', 'r']].to_numpy()

array([[1800,  353, 2301, 1346],
       [1747,  348, 2246, 1345],
       [1694,  344, 2192, 1344],
       [1641,  340, 2138, 1343],
       [1588,  336, 2084, 1342],
       [1535,  332, 2030, 1341],
       [1482,  327, 1975, 1340],
       [1429,  323, 1921, 1339],
       [1376,  319, 1867, 1338],
       [1323,  315, 1813, 1337],
       [1270,  311, 1759, 1337],
       [1221,  324, 1690, 1339],
       [1173,  338, 1621, 1342],
       [1124,  351, 1552, 1345],
       [1076,  365, 1484, 1348],
       [1029,  362, 1432, 1345],
       [ 983,  359, 1381, 1342],
       [ 936,  357, 1330, 1339],
       [ 890,  354, 1279, 1336],
       [ 844,  352, 1228, 1334],
       [ 806,  348, 1190, 1330],
       [ 768,  344, 1152, 1326],
       [ 730,  340, 1114, 1322],
       [ 693,  337, 1076, 1319],
       [ 660,  344, 1044, 1325],
       [ 628,  352, 1012, 1331],
       [ 596,  359,  980, 1338],
       [ 564,  367,  948, 1344],
       [ 532,  375,  916, 1351],
       [ 493,  378,  877, 1351],
       [ 4

In [19]:
df.iloc[7310]

frame          426.00000
id              16.00000
t             1800.00000
l              353.00000
b             2301.00000
r             1346.00000
conf             1.00000
class            1.00000
visibility       0.17653
Name: 7310, dtype: float64

In [12]:
df[df['class'] == 1][df['visibility'] < 0.5][df['id'] == 2]

C:\Users\meysam\AppData\Local\Temp\ipykernel_7992\1648902387.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['class'] == 1][df['visibility'] < 0.5][df['id'] == 2]
C:\Users\meysam\AppData\Local\Temp\ipykernel_7992\1648902387.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['class'] == 1][df['visibility'] < 0.5][df['id'] == 2]


,frame,id,t,l,b,r,conf,class,visibility
655,56,2,1822,385,198,498,1,1,0.49749


In [5]:
len(df)

30003

In [ ]:

df.to_csv('mix.txt')

In [ ]:
import pandas as pd
import os

# file_name = 'DanceTrack/train/dancetrack0001/file.txt'
# df = pd.read_csv(file_name, names=['frame', 'id', 'tg', 'lg', 'bg', 'rg', 'td', 'ld', 'bd', 'rd'])
# df = df.astype(int)

for dataset in 'DanceTrack', 'MOT17', 'MOT20':
    for split in ['train', 'val']:
        for seq in os.listdir(f'{dataset}/{split}'):
            file_name = f'{dataset}/{split}/{seq}/file.txt'
            df = pd.read_csv(file_name, names=['frame', 'id', 'tg', 'lg', 'bg', 'rg', 'td', 'ld', 'bd', 'rd'])
            df = df.astype(int)
            df.to_csv(f'{dataset}/{split}/{seq}/file.txt', header=None, index=None)
    #         break
    #     break
    # break

In [103]:
import pandas as pd
import os, shutil

# file_name = 'DanceTrack/train/dancetrack0001/file.txt'
# df = pd.read_csv(file_name, names=['frame', 'id', 'tg', 'lg', 'bg', 'rg', 'td', 'ld', 'bd', 'rd'])
# df = df.astype(int)

for dataset in 'DanceTrack', 'MOT17', 'MOT20':
    for split in ['train', 'val']:
        for seq in os.listdir(f'{dataset}/{split}'):
            os.makedirs(f'{dataset}/{split}/{seq}/gt')
            shutil.copy(f'../../.Datasets/{dataset}/{split}/{seq}/gt/gt.txt', f'{dataset}/{split}/{seq}/gt/gt.txt')
            shutil.copy(f'../../.Datasets/{dataset}/{split}/{seq}/seqinfo.ini', f'{dataset}/{split}/{seq}/seqinfo.ini')
            # file_name = f'{dataset}/{split}/{seq}/file.txt'
            # df = pd.read_csv(file_name, names=['frame', 'id', 'tg', 'lg', 'bg', 'rg', 'td', 'ld', 'bd', 'rd'])
            # df = df.astype(int)
            # df.to_csv(f'{dataset}/{split}/{seq}/file.txt', header=None, index=None)
    #         break
    #     break
    # break

In [92]:
float(cfp['Sequence']['imWidth'])

1920.0

In [22]:
import configparser

df = pd.read_csv('file.txt', names=['frame', 'id', 'tg', 'lg', 'bg', 'rg', 'td', 'ld', 'bd', 'rd'])
# df[['tg', 'bg', 'td', 'bd']].head(50)
df[df['id'] == 10][['td', 'ld', 'bd', 'rd']]

,td,ld,bd,rd
0,1090.125000,483.637512,1123.875000,597.037537
33,1090.125000,484.312531,1123.875000,597.037537
50,1092.150024,484.312531,1124.550049,596.362549
86,1091.474976,483.975006,1123.875000,596.025024
102,1091.474976,483.637512,1123.875000,596.362549
...,...,...,...,...
17677,1063.800049,454.612518,1102.950073,565.987549
17686,1063.125000,451.237518,1100.925049,565.987549
17711,1063.800049,451.912506,1101.599976,561.262512
17735,1063.125000,450.562500,1102.275024,561.262512


In [80]:
float(cfp['Sequence']['imWidth'])

1280.0

In [ ]:
df.clip()

In [62]:
print(dataset, split, seq)

DanceTrack train dancetrack0001


In [18]:
import numpy as np
import pandas as pd

df = pd.read_csv('DanceTrack/val/dancetrack0004/file.txt', names=['frame', 'id', 'tg', 'lg', 'bg', 'rg', 'td', 'ld', 'bd', 'rd'])
# df[['tg', 'lg', 'bg', 'rg']]
df[~((df['bg'] > 1920) | (df['rg'] > 1080) | (df['tg'] < 0) | (df['lg'] < 0))]
# np.abs(df[['td', 'ld', 'bd', 'rd']].to_numpy() - df[['tg', 'lg', 'bg', 'rg']].to_numpy()).reshape(-1).argmax()

,frame,id,tg,lg,bg,rg,td,ld,bd,rd
0,1,1,956,434,1258,992,969,414,1269,993
1,1,2,624,380,833,1008,624,379,847,1009
2,1,3,796,315,1036,1079,772,309,1034,1077
3,2,2,605,385,834,1014,607,385,843,1022
4,2,3,746,318,1040,1079,736,316,1041,1077
...,...,...,...,...,...,...,...,...,...,...
4492,1202,2,680,467,918,796,683,466,920,796
4493,1203,0,1115,420,1406,879,1115,419,1409,878
4494,1203,1,865,450,1119,829,866,448,1119,827
4495,1203,2,679,458,904,789,676,456,904,787


In [16]:
df[df['id'] == 8].head(50)

,frame,id,tg,lg,bg,rg,td,ld,bd,rd
11,1,8.0,1416.0,431.0,1600.0,767.0,1418.850098,430.650024,1594.350098,759.375000
30,2,8.0,1422.0,431.0,1605.0,768.0,1424.250000,432.000000,1597.050049,763.425049
62,3,8.0,1428.0,431.0,1610.0,769.0,1429.650024,429.975006,1613.250000,766.125000
68,4,8.0,1434.0,431.0,1615.0,770.0,1433.700073,428.625000,1614.600098,766.125000
100,5,8.0,1440.0,432.0,1620.0,771.0,1443.150024,428.625000,1615.950073,766.800049
111,6,8.0,1454.0,431.0,1626.0,772.0,1453.950073,426.600006,1624.050049,768.825012
151,7,8.0,1468.0,430.0,1633.0,773.0,1468.800049,427.275024,1628.100098,768.825012
159,8,8.0,1482.0,430.0,1640.0,774.0,1483.650024,423.900024,1640.250000,770.175049
182,9,8.0,1497.0,429.0,1646.0,775.0,1499.850098,423.225006,1651.050049,770.175049
203,10,8.0,1511.0,429.0,1653.0,776.0,1518.750000,423.225006,1669.950073,773.550049


In [1]:
import pandas as pd

df = pd.read_csv('file.txt')

,1,10.0,1091.0,484.0,1122.0,599.0,1090.125,483.6375122070313,1123.875,597.0375366210938
0,1,69.0,596.0,429.0,614.0,471.0,595.687500,429.975006,612.562500,469.800018
1,1,19.0,442.0,446.0,547.0,729.0,440.100006,445.500000,546.075012,723.600037
2,1,17.0,1101.0,441.0,1139.0,549.0,1099.575073,438.412506,1137.375000,548.437500
3,1,3.0,586.0,447.0,671.0,710.0,585.225037,446.850006,671.625000,708.075012
4,1,14.0,1255.0,447.0,1288.0,547.0,1254.825073,447.862518,1291.275024,547.762512
...,...,...,...,...,...,...,...,...,...,...
17761,600,18.0,926.0,435.0,969.0,549.0,926.100037,449.212524,958.500061,528.187500
17762,600,17.0,985.0,451.0,1025.0,550.0,984.825012,449.550018,1019.925049,546.750000
17763,600,15.0,1004.0,443.0,1043.0,551.0,1002.375061,451.912506,1034.775024,545.062500
17764,600,10.0,1061.0,445.0,1104.0,567.0,1064.474976,445.162506,1104.974976,567.337524


In [71]:
df_ = pd.read_csv('../../.Datasets/MOT17/train/MOT17-02-FRCNN/gt/gt.txt', names=['frame', 'id', 't', 'l', 'b', 'r', 'conf', 'class', 'visibility'])

df_[df_['id'] == 16]

,frame,id,t,l,b,r,conf,class,visibility
7310,426,16,1800,353,501,993,1,1,0.17653
7311,427,16,1747,348,499,997,1,1,0.25560
7312,428,16,1694,344,498,1000,1,1,0.33493
7313,429,16,1641,340,497,1003,1,1,0.41497
7314,430,16,1588,336,496,1006,1,1,0.49570
...,...,...,...,...,...,...,...,...,...
7367,483,16,-263,382,416,952,1,1,0.26912
7368,484,16,-286,382,417,952,1,1,0.22987
7369,485,16,-308,382,416,952,1,1,0.18996
7370,486,16,-331,382,417,952,1,1,0.15091


In [45]:
len(df)

17767

In [65]:
np.abs(df[['td', 'ld', 'bd', 'rd']].to_numpy() - df[['tg', 'lg', 'bg', 'rg']].to_numpy()).reshape(-1)[50275]

np.float64(418.10009765625)

In [47]:
50275 // 4

12568

In [66]:
df[df['id'] == 16][['tg', 'lg', 'bg', 'rg']].astype(int).head(50)

,tg,lg,bg,rg
12530,1800,353,2301,1346
12568,1747,348,2246,1345
12607,1694,344,2192,1344
12658,1641,340,2138,1343
12687,1588,336,2084,1342
12726,1535,332,2030,1341
12737,1482,327,1975,1340
12772,1429,323,1921,1339
12800,1376,319,1867,1338
12855,1323,315,1813,1337


In [67]:
df.iloc[12568]

frame     427.000000
id         16.000000
tg       1747.000000
lg        348.000000
bg       2246.000000
rg       1345.000000
td       1728.000000
ld        286.875000
bd       2219.400146
rd       1763.100098
Name: 12568, dtype: float64

In [49]:
(df[['td', 'ld', 'bd', 'rd']].to_numpy() - df[['tg', 'lg', 'bg', 'rg']].to_numpy())[12568]

array([-19.        , -61.125     , -26.59985352, 418.10009766])

In [72]:
df[['td', 'ld', 'bd', 'rd']].to_numpy()[12568]

array([1728.        ,  286.875     , 2219.40014648, 1763.10009766])

In [73]:
df[['tg', 'lg', 'bg', 'rg']].to_numpy()[12568]

array([1747.,  348., 2246., 1345.])

In [ ]:
import pandas as pd
base = '../../.Datasets/'
base_dets = '../../.Detections/'

for dataset in 'DanceTrack', 'MOT17', 'MOT20':
    for split in ['train', 'val', 'test']:
        os.remove(f'{base_dets}/{dataset}/{split}/det.txt')
        for seq in os.listdir(f'{base}/{dataset}/{split}'):
            os.makedirs(f'{base_dets}/{dataset}/{split}/{seq}', exist_ok=True)
            
            shutil.copy(f'{base_dets}/{dataset}/{seq}.txt', f'{base_dets}/{dataset}/{split}/{seq}/det.txt')

In [24]:
import shutil, os

base = '../../.Datasets/'
base_dets = '../../.Detections/'

for dataset in 'DanceTrack', 'MOT17', 'MOT20':
    for split in ['train', 'val', 'test']:
        os.remove(f'{base_dets}/{dataset}/{split}/det.txt')
        for seq in os.listdir(f'{base}/{dataset}/{split}'):
            os.makedirs(f'{base_dets}/{dataset}/{split}/{seq}', exist_ok=True)
            
            shutil.copy(f'{base_dets}/{dataset}/{seq}.txt', f'{base_dets}/{dataset}/{split}/{seq}/det.txt')



In [19]:
import pandas as pd

f = pd.DataFrame([[1,2]], columns=['a', 'b'])
pd.concat([f, pd.DataFrame([[3,4]], columns=['a', 'b'])])
# len(f)
# f

,a,b
0,1,2
0,3,4
